In [777]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
import pickle
import keras
from keras_visualizer import visualizer 
from keras.models import Model
import tensorflow as tf
import tensorflow.math as math
import keras.layers as layers
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [782]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, hop_length=1024, n_mfcc=20)
    
    return mfccs.flatten()

In [783]:
def analysis_data_frame(parameters):
    idx = int(parameters[0])
    print(idx, end="\r")
    synth_parameters = parameters[1:]
    wav = "/home/luc/src/python/fm_tensor/recs3/fm" + str(idx) + ".wav"
    feature = extract_features(wav);
    
    return feature, synth_parameters

In [784]:
def analyze_audios():
    specs = np.genfromtxt('/home/luc/src/python/fm_tensor/recs3/specs.csv', delimiter=',')
    frames = list(map(analysis_data_frame, specs))
    data_frames = pd.DataFrame(frames, columns=['feature','label'])

    return data_frames

In [785]:
data = analyze_audios()

In [776]:
specs = np.genfromtxt('/home/luc/src/python/fm_tensor/recs3/specs.csv', delimiter=',')
data['label'] = specs.to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [786]:
data.label[0]

array([46.64231157,  0.5       ,  1.10478791,  0.01036095])

In [787]:
file = open('/home/luc/src/python/fm_tensor/recs3/data.pkl', 'wb')
pickle.dump(data, file)
file.close()

In [788]:
 data.head()

,feature,label
0,"[-514.0832, -546.2257, -547.7809, -548.06384, ...","[46.642311573029, 0.5, 1.1047879099846, 0.0103..."
1,"[-513.8092, -545.7849, -550.1052, -550.35986, ...","[41.149642467499, 0.5, 1.122685790062, 0.01617..."
2,"[-517.2228, -545.9017, -550.81244, -549.3952, ...","[39.642944812775, 0.6, 1.0858309864998, 0.0337..."
3,"[-516.4214, -543.3951, -546.614, -546.53217, -...","[44.028305053711, 0.5, 1.0036882281303, 0.0492..."
4,"[-497.70135, -541.6419, -546.3492, -545.2183, ...","[37.099526882172, 0.5, 1.0231810927391, 0.0994..."


In [793]:
# normalizing y (the synth parameters)
X = np.array(data.feature.tolist())
y_not_normalized = np.array(data.label.tolist())
y = y_not_normalized / y_not_normalized.max(axis= 0)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 127)

In [794]:
y_train

array([[0.65468709, 0.07194245, 0.58046454, 0.00700735],
       [0.01761434, 0.68345324, 0.57290597, 0.05564194],
       [0.05194696, 0.08633094, 0.33965695, 0.20006298],
       ...,
       [0.02052235, 0.44604317, 0.02314291, 0.00767145],
       [0.00497319, 0.14388489, 0.23213065, 0.18503013],
       [0.03957119, 0.10071942, 0.07438449, 0.18873632]])

In [822]:
def build_model_graph(input_shape=(440,), n_outputs=6):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(60))
    model.add(Activation('relu'))
  #  model.add(Dropout(0.1))
    model.add(Dense(40))
    model.add(Activation('relu'))
  #  model.add(Dropout(0.1))
    model.add(Dense(20))
    model.add(Activation('relu'))
   # model.add(Dropout(0.1))
    model.add(Dense(n_outputs))
    #model.add(Activation('softmax'))
    # Compile the model
    #model.compile(loss='mean_absolute_error', metrics=['accuracy'], optimizer='adam')
    rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(loss='mse', metrics=[rmse], optimizer='adam')

    return model

In [823]:
model = build_model_graph((x_train.shape[1],), y_train.shape[1])
model.summary()

Model: "sequential_84"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_246 (Dense)            (None, 160)               70560     
_________________________________________________________________
activation_97 (Activation)   (None, 160)               0         
_________________________________________________________________
dense_247 (Dense)            (None, 40)                6440      
_________________________________________________________________
activation_98 (Activation)   (None, 40)                0         
_________________________________________________________________
dense_248 (Dense)            (None, 20)                820       
_________________________________________________________________
activation_99 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_249 (Dense)            (None, 4)               

In [824]:
# Calculate pre-training accuracy 
model.evaluate(x_test, y_test, verbose=0)

[2194.386962890625, 46.84428405761719]

In [825]:
num_epochs = 1000
num_batch_size = 32

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

Epoch 1/1000
1500/1500 [==============================] - 1s 525us/step - loss: 5.0339 - root_mean_squared_error: 2.2436 - val_loss: 0.0642 - val_root_mean_squared_error: 0.2534
Epoch 2/1000
1500/1500 [==============================] - 1s 523us/step - loss: 0.0631 - root_mean_squared_error: 0.2512 - val_loss: 0.0616 - val_root_mean_squared_error: 0.2481
Epoch 3/1000
1500/1500 [==============================] - 1s 524us/step - loss: 0.0606 - root_mean_squared_error: 0.2462 - val_loss: 0.0589 - val_root_mean_squared_error: 0.2426
Epoch 4/1000
1500/1500 [==============================] - 1s 526us/step - loss: 0.0566 - root_mean_squared_error: 0.2380 - val_loss: 0.0557 - val_root_mean_squared_error: 0.2359
Epoch 5/1000
1500/1500 [==============================] - 1s 528us/step - loss: 0.0544 - root_mean_squared_error: 0.2333 - val_loss: 0.0546 - val_root_mean_squared_error: 0.2337
Epoch 6/1000
1500/1500 [==============================] - 1s 525us/step - loss: 0.0549 - root_mean_squared_err

Epoch 47/1000
1500/1500 [==============================] - 1s 610us/step - loss: 0.0532 - root_mean_squared_error: 0.2306 - val_loss: 0.0547 - val_root_mean_squared_error: 0.2338
Epoch 48/1000
1500/1500 [==============================] - 1s 572us/step - loss: 0.0536 - root_mean_squared_error: 0.2314 - val_loss: 0.0542 - val_root_mean_squared_error: 0.2327
Epoch 49/1000
1500/1500 [==============================] - 1s 547us/step - loss: 0.0535 - root_mean_squared_error: 0.2314 - val_loss: 0.0538 - val_root_mean_squared_error: 0.2319
Epoch 50/1000
1500/1500 [==============================] - 1s 544us/step - loss: 0.0555 - root_mean_squared_error: 0.2356 - val_loss: 0.0559 - val_root_mean_squared_error: 0.2365
Epoch 51/1000
1500/1500 [==============================] - 1s 543us/step - loss: 0.0540 - root_mean_squared_error: 0.2324 - val_loss: 0.0526 - val_root_mean_squared_error: 0.2294
Epoch 52/1000
1500/1500 [==============================] - 1s 611us/step - loss: 0.0524 - root_mean_squar

KeyboardInterrupt: 

In [802]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training score: {0:.2}".format(score[1]))

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing score: {0:.2}".format(score[1]))

Training score: 0.15
Testing score: 0.21


In [837]:
#prediction = model.predict(X)
#X
np.set_printoptions(suppress=True)
print(model.predict(x_train[1517:1518]))
print(y_train[1517:1518])

[[0.16114974 0.21437457 0.49448773 0.05972648]]
[[0.10609904 0.15107914 0.11010481 0.17402033]]


In [736]:
model.evaluate(x_train, y_train)

391/391 [==============================] - 1s 1ms/step - loss: 0.1484 - root_mean_squared_error: 0.3852


[0.1484052538871765, 0.385233998298645]

In [833]:
x_train[517:518]

array([[-214.36975   , -188.01505   , -183.9505    , -190.17633   ,
        -186.16972   , -185.95804   , -189.28755   , -181.80202   ,
        -183.95082   , -190.37267   , -192.83992   , -186.38098   ,
        -186.8587    , -186.25699   , -189.98299   , -182.99745   ,
        -181.37186   , -189.56105   , -211.05821   , -229.34169   ,
        -270.6906    , -338.3115    ,  327.65057   ,  333.45898   ,
         332.72922   ,  331.35632   ,  333.35507   ,  334.82996   ,
         334.17682   ,  333.01807   ,  335.08136   ,  335.1153    ,
         335.00397   ,  333.53152   ,  333.02942   ,  331.0492    ,
         336.83127   ,  329.53702   ,  334.39966   ,  329.79742   ,
         327.0368    ,  315.3733    ,  299.12393   ,  264.03003   ,
         -60.08419   ,  -75.58531   ,  -79.80492   ,  -76.690506  ,
         -78.75038   ,  -78.718796  ,  -74.43533   ,  -80.05696   ,
         -79.34136   ,  -71.65438   ,  -70.83493   ,  -78.909676  ,
         -78.3525    ,  -77.63565   ,  -71.54489

In [801]:
model.save('/home/luc/src/python/fm_tensor/recs3/model_60_40_20.model')

2021-10-20 17:25:04.415099: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/luc/src/python/fm_tensor/recs3/model_60_40_20.model/assets


In [826]:
model = keras.models.load_model('/home/luc/src/python/fm_tensor/recs3/model_60_40_20.model')

In [844]:
def predict_audio(file):
    features = extract_features(file)
    prediction = model.predict(np.array([features]))[0]
    maxs = np.array(data.label.tolist()).max(axis=0)
    scaled = prediction * maxs
    
    return scaled

In [867]:
predict_audio('/home/luc/src/python/fm_tensor/whistle.wav')

array([-209.72848826,    1.31718638,    1.31537357,    0.05041321])